In [1]:
from aocd import get_data

# Day 8: Resonant Collinearity

## Part One

Given a 2D grid representing antennas with specific frequencies (letters, digits), identify locations called "antinodes" where signals resonate.

Rules for Antinodes:

**Same Frequency:** Two antennas must have the same frequency for there to be antinodes

**Collinearity:** The antennas and antinodes lie on a straight line

**Resonance Condition:** One antenna must be twice as far from the antinode as the other

**Antinode Count:** Each valid antenna pair generates two antinodes (one on each side)

**Objective:** Count the unique antinode locations within the grid bounds

In [2]:
example_input = """
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
"""

In [3]:
import numpy as np

example_grid = np.array([list(line) for line in example_input.strip().split("\n")]); example_grid

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')

In [4]:
example_grid.shape

(12, 12)

Identify the positions of all antennae with their corresponding "frequency" values

In [5]:
def parse_grid(grid):

    # boolean mask for alphanumeric elements
    mask = np.char.isalnum(grid)
    
    positions = np.argwhere(mask).astype(np.int64)
    antenna_values = grid[mask]

    # can't use np.column_stack because it would turn `positions` into strings to match `antenna_values` type
    # so here's a workaround
    antennae = np.empty((positions.shape[0], 3), dtype=object)
    antennae[:, :2] = positions
    antennae[:, 2] = antenna_values
    
    return antennae

In [6]:
antennae = parse_grid(example_grid); antennae

array([[1, 8, '0'],
       [2, 5, '0'],
       [3, 7, '0'],
       [4, 4, '0'],
       [5, 6, 'A'],
       [8, 8, 'A'],
       [9, 9, 'A']], dtype=object)

- Loop through all possible pairs of antennas that have the same frequency
- For each pair, find the potential antinode locations such that one antenna is twice as far from the antinode as the other
- Ensure that the calculated antinode locations are within the bounds of the map
- Use a `set` to store and deduplicate the valid antinode locations
- Return the number of unique locations stored in the set

#### First Approach

In [7]:
def within_bounds(node, grid):
    rows, cols = grid.shape
    return 0 <= node[0] < rows and 0 <= node[1] < cols

In [8]:
antinodes = set()
    
for i in range(len(antennae)):
    for j in range(i + 1, len(antennae)):
        r1, c1, val1 = antennae[i]
        r2, c2, val2 = antennae[j]
        
        if val1 != val2:
            continue
        
        # deltas for calculating antinode locations
        dr, dc = r2 - r1, c2 - c1
        
        # potential antinode locations
        antinode1 = (r1 - dr, c1 - dc)
        antinode2 = (r2 + dr, c2 + dc)
        
        # add valid antinodes that are within bounds
        if within_bounds(antinode1, example_grid):
            antinodes.add(antinode1)
        if within_bounds(antinode2, example_grid):
            antinodes.add(antinode2)

In [9]:
len(antinodes)

14

#### Vecotorized

In [10]:
antennae

array([[1, 8, '0'],
       [2, 5, '0'],
       [3, 7, '0'],
       [4, 4, '0'],
       [5, 6, 'A'],
       [8, 8, 'A'],
       [9, 9, 'A']], dtype=object)

In [11]:
def find_antinodes(antennae, grid_shape):
    
    coords = antennae[:, :2]
    antenna_values = antennae[:, -1]
    
    # create all unique pairs of indices (i, j) where i < j with this one weird trick
    i_indices, j_indices = np.triu_indices(len(antennae), k=1)
    
    coords1 = coords[i_indices]
    coords2 = coords[j_indices]
    vals1 = antenna_values[i_indices]
    vals2 = antenna_values[j_indices]
    
    matching_pairs = vals1 == vals2
    
    dr_dc = coords2[matching_pairs] - coords1[matching_pairs]
    
    # potential antinodes
    antinode1 = coords1[matching_pairs] - dr_dc
    antinode2 = coords2[matching_pairs] + dr_dc
    
    def within_bounds(coords, shape):
        return np.all((coords >= 0) & (coords < shape), axis=1)
    
    valid_antinode1 = antinode1[within_bounds(antinode1, grid_shape)]
    valid_antinode2 = antinode2[within_bounds(antinode2, grid_shape)]
    
    # combine and remove duplicates by converting to set of tuples
    antinodes = set(map(tuple, valid_antinode1)) | set(map(tuple, valid_antinode2))
    
    return antinodes


In [12]:
len(find_antinodes(antennae, example_grid.shape))

14

In [13]:
data = get_data(day=8, year=2024)

In [14]:
grid = np.array([list(line) for line in data.strip().split("\n")])

In [15]:
antennae = parse_grid(grid); antennae[:10]

array([[0, 21, '5'],
       [0, 37, 'P'],
       [1, 13, 'w'],
       [1, 19, 'T'],
       [1, 29, 'X'],
       [1, 30, 'h'],
       [1, 36, '5'],
       [1, 49, 'u'],
       [2, 19, 'k'],
       [2, 20, 'X']], dtype=object)

In [16]:
len(find_antinodes(antennae, grid.shape))

400

## Part Two

The extension in Part 2 introduces a new concept of *resonant harmonics*, which expands the way antinodes are determined compared to the original problem. 

An antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance.

The three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes.

In [17]:
new_example = """
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
"""

In [18]:
new_grid = np.array([list(line) for line in new_example.strip().split("\n")])

In [19]:
parse_grid(new_grid)

array([[0, 0, 'T'],
       [1, 3, 'T'],
       [2, 1, 'T']], dtype=object)

What if we created a generator to provide antinodes instead:

In [20]:
def antinode_generator(grid):

    def _generator(node_1, node_2):
        r1, c1 = node_1
        r2, c2 = node_2
        dr, dc = r2 - r1, c2 - c1

        node = node_1
        while within_bounds(node, grid):
            yield node
            node = node[0] - dr, node[1] - dc

        node = node_2
        while within_bounds(node, grid):
            yield node
            node = node[0] + dr, node[1] + dc

    return _generator

In [21]:
def find_antinodes(antennae, grid):
    gen = antinode_generator(grid)
    antinodes = []
    antenna_values = set(antennae[:, -1])

    for i in range(len(antennae)):
        for j in range(i + 1, len(antennae)):
            r1, c1, val1 = antennae[i]
            r2, c2, val2 = antennae[j]
        
            if val1 != val2:
                continue

            antinodes.extend(node for node in gen((r1,c1), (r2,c2)))
    
    return antinodes

In [22]:
def get_antinodes(grid):
    antinodes = set()
    antennae = parse_grid(grid)
    
    for i, j in find_antinodes(antennae, grid):
        antinodes.add((i,j))

    return antinodes
        

In [23]:
get_antinodes(new_grid)

{(0, 0), (0, 5), (1, 3), (2, 1), (2, 6), (3, 9), (4, 2), (6, 3), (8, 4)}

In [24]:
len(get_antinodes(new_grid))

9

In [25]:
len(get_antinodes(grid))

1280